<a href="https://colab.research.google.com/github/B-Lorentz/physics-bsc-visual/blob/main/15_Quantum_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from jax import numpy as jnp
from jax import grad, vmap, jit, random, lax, ops
from matplotlib.widgets import Slider, Button, RadioButtons
from ipywidgets import interact, widgets

# Quantum Statistics

### Ideal quantum gases: Bose-Einstein and Fermi-Dirac statistics

A key property of quantum mechanics is that identical particles are considered "the same": There are no labels attached. But our wavefunctions distinguish them merely by having variables specified in an ordered manner: $$\Psi(x_1, x_2, x_3,...)$$
(Here $x$ can include spin or any other descriptors)

To remedy this, the wavefunction must be either symmetric or anti-symmetric in the interchange of particles:
- symmetric: $\Psi(x_1, x_2, x_3,...) = \Psi(x_2, x_1, x_3,...) = \Psi(x_2, x_3, x_1,...) = ...$

- anti-symmtteric: $\Psi(x_1, x_2, x_3,...) = -\Psi(x_2, x_1, x_3,...) = \Psi(x_2, x_3, x_1,...) = ...$

This way repeated interchange of two perticles is the identity as expected ($1 = 1\cdot1 = (-1)\cdot(-1)$), but since the probabilities are given by $|\Psi|^2$, the physical state is unchanged under permutation even in the antisymmetric case. ($\cdot(-1)$ is just a phase change by $\pi$ )

It can be shown that whether the symmetric or the anti-symmetric case is in force depends only on the particle type. Particles whose wavefunction is symmetric to interchange are called bosons, and the anti-symmetric ones fermions. 

It is an additional important fact (the spin-statistics theorem) that bosons have integer spins and fermions half-integer spins. This is however not part of their definition, but a separate and notrivial result that can be arrived at from relativistc field theory.

If the particles are non-interacting (their Hamiltionian is the sum of the individual Hamiltionians)  we can construct states with the required symmetry by linearly combining products of one-particle states:

- For bosons all permutations are simply added together. Two particles: $\Psi(x_1, x_2) = \Psi_1(x_1)\Psi_2(x_2) +  \Psi_1(x_2)\Psi_2(x_1)$

- For fermions we multiply them by $-1^p$ where $p$ is the number of permutations needed from the natural ordering. Two particles: $\Psi(x_1, x_2) = \Psi_1(x_1)\Psi_2(x_2) - \Psi_1(x_2)\Psi_2(x_1)$. Therefore if $\Psi_1 \equiv \Psi_2$, the wavefunction is zero everywhere, prohibiting two fermions from ever assuming the same quantum state. THis is called the Pauli exclusion principle. (This generalized to $N$ particles where the wavefunction can be written asa  determinat formed from the one-particle wavefunctions)

A demonstration for two particles in a 1D infinite potential well:

In [7]:
x1 = x2 = np.linspace(0, 1, 100)

def ploti(name, f):
  x1s, x2s = np.meshgrid(x1, x2)
  plt.contourf(x1, x2, f(x1s, x2s), levels=50,cmap=plt.get_cmap("coolwarm"))
  plt.plot([0,1],[0,1], "black")
  #plt.plot(x1, psi1(x1))
  plt.xlabel("$x_1$", fontsize=14)
  plt.xlabel("$x_2$", fontsize=14)
  plt.title(name, fontsize=16)
def interplot(n1, n2):
  psi1 = lambda x: jnp.sin(n1*np.pi*x)
  psi2 = lambda x: jnp.sin(n2*np.pi*x)
  plt.figure(figsize=(16, 8))
  plt.subplot(121)
  ploti("Bozon", lambda x1, x2: psi1(x1)*psi2(x2) + psi1(x2)*psi2(x1))

  plt.subplot(122)
  ploti("Fermion", lambda x1, x2: psi1(x1)*psi2(x2) - psi1(x2)*psi2(x1))
interact(interplot, n1=widgets.IntSlider(1, 1, 10), n2=widgets.IntSlider(3, 1, 10))


interactive(children=(IntSlider(value=1, description='n1', max=10, min=1), IntSlider(value=3, description='n2'…

<function __main__.interplot>

To do statistical physics with non-interaction identical particles - that is, to study an ideal gas made of fermions or bosons - we use the occupation representation: For each quantum state with energy $\epsilon_i$ we record only the number of particles $n_i$, and then compute the grand canonical partition function as $$\mathcal{Z} = \sum_{n_1}\sum_{n_2}...\prod_{k=1}^{\infty}e^{-\beta n_k(\epsilon_k -\mu)}$$

For bosons $n_i\in\mathbb{N}$, while for fermions $n_i \in \{0, 1\}$, encoding the only difference between them, the Pauli exclusion.

This can be computed by cleverly interchanging the sum and the product and solving the sum (which is trivial for the fermions and a geometric series for bosons, that converges if $\mu<\epsilon_0$).

$\langle n_i \rangle$ van be computed by noticing that in:
$$\langle n_i \rangle = \frac{1}{\mathcal{Z}}\sum_{n_1}\sum_{n_2}...n_i\prod_{k=1}^{\infty}e^{-\beta n_k(\epsilon_k -\mu)}$$ the respective contributions of all energy levels other than the $i$th are destroyed by the same appearing in $\mathcal{Z}$.

The result is:

- $\langle n_i \rangle = f_B(\epsilon_i) = \frac{1}{e^{\beta(\epsilon_i - \mu) }-1}$ for bosons.

- $\langle n_i \rangle = f_F(\epsilon_i) = \frac{1}{e^{\beta(\epsilon_i - \mu)  }+1}$ for fermions.

The following interactive plots illustrate the shape of these two functions, and also show how there is nothing more ongoing than grand-canonical formalism in occupancy representation, as I show an approximate computation where I simply sample {n_i}-s (like (n_1=1, n_2=3, n_3=0 ...)) using a Monte-Carlo walk with grand canonical probabilities.

### High temperature limit

For a sytem with a fixed number ($N$) of particles (like a gas in a box, but not a solid with phonons or a box with photons) $\mu$ is given implicitly by:
$$N=\sum_i f(\epsilon_i, \mu)= \sum_i \frac{1}{e^{\beta(\epsilon_i - \mu) }\pm1}$$

If $T \to \infty \iff \beta\to0$

therefore $\mu<<0 \implies e^{\beta(\epsilon_i - \mu)} >> 1$

Bosons and fermions all comverge to the Boltzmann-distribution.

In [3]:
def MCn(Es, beta, mu, stat):
  N_es = Es.shape[0]
  efun = lambda ns: (ns*(Es-mu)).sum()
  def mcf(key):
    def upd(i, state):
      ns, sum_ns, key = state
      k1, k2, k3, k4 = random.split(key, 4)
      inds = random.choice(k1, N_es, shape=(N_es//20,), replace=False)
      ns2 = ns.at[inds].set(stat.proposer(k2, ns[inds]))
      e0 = efun(ns)
      e1 = efun(ns2)
      dec = jnp.logical_or(e1<e0, jnp.exp(-beta*(e1-e0)) > random.uniform(k3, (1,)))
      ns = lax.cond(dec[0], lambda x:ns2, lambda x:x, ns)
      sum_ns = lax.cond(i%5 == 4, lambda x: x+ns, lambda x:x, sum_ns)
      return (ns, sum_ns, k4)
    k1, k2 = random.split(key)
    init = (stat.proposer(k1, np.zeros(N_es, dtype=int)), np.zeros(N_es, dtype=int), k2)
    res = lax.fori_loop(0, 1000, upd, init)[1]
    return res
  return mcf
class Stat:
  def __init__(self, name, f, proposer):
    self.name = name
    self.f = f
    self.proposer = proposer
  
fermi = Stat("F", lambda E, beta, mu : 1/(np.exp(beta*(E-mu))+1), lambda key, ns: random.choice(key, 2, shape=ns.shape ))

In [4]:
N_stat = 100
Es = np.linspace(0, 1.0, N_stat)

def plotbuilder(stat):
  def plotter(beta, mu):
    plt.figure(figsize=(10, 8))
    ns = stat.f(Es, beta, mu)
    plt.plot(Es, ns, label=f"$f_{stat.name}(\epsilon)$")
    bns = np.exp(-beta*Es)
    plt.plot(Es, bns*ns.max(), label="Boltzmann")
    plt.plot([mu, mu], [0, ns.max()], "r", label=r"$\mu$")
    plt.xlabel("$\epsilon$", fontsize=14)
    plt.ylabel("<n>", fontsize=14)
    plt.grid()
    mcs = vmap(MCn(Es, beta, mu, stat), 0)(random.split(random.PRNGKey(127), 100))
    res, resd = mcs.mean(0), mcs.std(0)
    maxi = res.max()/ns.max()
    #print(res.shape, res.min(), res.max())
    plt.errorbar(Es, res/maxi, resd/maxi)
    plt.title(r"$\beta =" +f"{beta:.2f}$")
  
    plt.legend(fontsize=14)
  return plotter
  
interact(plotbuilder(fermi), beta=widgets.FloatSlider(min=1.0, max=30, value=3.5), mu=widgets.FloatSlider(min=-1, max=1, value=0.1, step=0.05))
None


interactive(children=(FloatSlider(value=3.5, description='beta', max=30.0, min=1.0), FloatSlider(value=0.1, de…

In [5]:
bose = Stat("B", lambda E, beta, mu : 1/(np.exp(beta*(E-mu))-1), lambda key, ns: jnp.maximum((random.normal(key, ns.shape)*jnp.maximum(ns, 10)/2).astype(int)+ns, 0) )
interact(plotbuilder(bose), beta=widgets.FloatSlider(min=1.0, max=10, value=1.5), mu=widgets.FloatSlider(min=-1, max=-0.15, value=-0.05, step=0.05))
None

interactive(children=(FloatSlider(value=1.5, description='beta', max=10.0, min=1.0), FloatSlider(value=-0.15, …

## Example: electrons in metals

$$D(\epsilon) = 4\pi V (\frac{2 m}{h^2})^{3/2} \sqrt{e}$$

at T=0: 

$$N = \int_0^{\epsilon_F}D(\epsilon)d\epsilon$$

$$\epsilon_F = \frac{h^2}{2m}\left(\frac{3 N}{8\pi V}\right)^{\frac{2}{3}}$$

$$E = \int_0^\infty\epsilon f(\epsilon) D(\epsilon)d \epsilon \approx  \int_0^{\epsilon_F}\epsilon D(\epsilon)d\epsilon$$

A Sommerfeld-series would give a better approximation

In [6]:
def plotbuilder2(stat):
  N_stat = 300
  Es = np.linspace(0, 2.0, N_stat)
  def plotter(beta):
    mu=1.0
    plt.figure(figsize=(10, 8))
    ns = stat.f(Es, beta, mu)*np.sqrt(Es)
    plt.plot(Es, ns, label="")
    bns = np.exp(-beta*Es)
    
    plt.plot([mu, mu], [0, ns.max()], "r", label=r"$\mu$")
    plt.xlabel("$\epsilon$", fontsize=14)
    plt.ylabel(r"$D(\epsilon)\cdot f(\epsilon)$", fontsize=14)
    plt.grid()
    plt.title(r"$\beta =" +f"{beta:.2f}$")
  
    plt.legend(fontsize=14)
  return plotter
  
interact(plotbuilder2(fermi), beta=widgets.FloatSlider(min=1.0, max=100, value=10))
None

interactive(children=(FloatSlider(value=10.0, description='beta', min=1.0), Output()), _dom_classes=('widget-i…